# ChatBot

Jeffrey Leiva Cascante 2021016720

Richard León Chinchilla 2019003759

# Imports necesarios y setup básico

In [36]:
import numpy as np
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import os
import string
import json
from sklearn.metrics.pairwise import cosine_similarity

nlp = spacy.load("en_core_web_lg")

# Lista de respuestas del chatbot
answers = [
    "Our business hours are Monday to Friday from 9:00 AM to 6:00 PM.",
    "You can make returns within the first 30 days of purchase.",
    "We accept credit cards, PayPal, and bank transfers.",
    "We ship nationwide. Delivery time varies depending on the location.",
    "We offer a wide range of products available in our online store.",
    "We offer warranties of up to 2 years on selected products.",
    "For technical support, you can contact us through our web form.",
    "You can place an order by following the steps on our shopping page.",
    "For any questions, you can contact us through our chat or email.",
    "To check the status of your order, go to the 'My Orders' section."
]

file_name = 'answers_embeddings.json'



# Definición de funciones para el funcionamiento del chatbot

In [37]:

def preprocess_text(text):
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc if token.text not in STOP_WORDS and token.text not in string.punctuation]
    return " ".join(tokens)

# Función para obtener el embedding de un texto utilizando
def get_embedding(text):
    return nlp(text).vector

# Función para guardar los embeddings en JSON
def save_embeddings_json(embeddings, name_file):
    with open(name_file, 'w') as f:
        json.dump(embeddings, f)

# Función para cargar embeddings desde un archivo JSON
def load_embeddings_json(name_file):
    with open(name_file, 'r') as f:
        embeddings_list = json.load(f)
    return [np.array(embedding) for embedding in embeddings_list]

# Función para calcular la similitud de coseno entre la pregunta y las respuestas
def calculate_similarity(embedding_question, embeddings_answers):
    # Calcula la similitud de coseno entre la pregunta y cada respuesta
    similarities = cosine_similarity([embedding_question], embeddings_answers)
    return similarities[0]

# Función para seleccionar la mejor respuesta en función de la similitud
def pick_best_answer(similarities):
    index_best_answer = np.argmax(similarities)  # Índice de la respuesta más similar
    return index_best_answer, max(similarities)


# Revisar si ya existen los embeddings de las respuestas

Si no existen en un archivo .json entonces los crea y almacena, si ya existen, simplemente los carga.

In [39]:

embeddings_respuestas = []
if os.path.exists(file_name):
    print("El archivo de embeddings ya existe. Cargando embeddings...")
    embeddings_respuestas = load_embeddings_json(file_name)
else:
    print("El archivo de embeddings no existe. Generando embeddings...")
    # Generar los embeddings de las respuestas
    preprocessed_answers = [preprocess_text(answer) for answer in answers]
    embeddings_respuestas = [get_embedding(answer) for answer in preprocessed_answers]
    
    # Guardar los embeddings en el archivo JSON
    embeddings_respuestas_list = [embedding.tolist() for embedding in embeddings_respuestas]
    save_embeddings_json(embeddings_respuestas_list, file_name)
    print(f"Embeddings generados y guardados en {file_name}.")



El archivo de embeddings ya existe. Cargando embeddings...


# Testear con una pregunta y ver la respuesta que tiene una mayor similitud

In [40]:

#Obtener la respuesta del usuario y hacerle el preprocesamiento
question = input("Haz tu pregunta: ")
preprocessed_question = preprocess_text(question)

# Obtener el emebedding de la pregunta
embedding_question = get_embedding(preprocessed_question)

# Calcular la similitud entre pregunta y respuestas
similarities = calculate_similarity(embedding_question, embeddings_respuestas)

# Seleccionar la mejor respuesta basandose en la similitud
index_best_answer, maximum_similarity = pick_best_answer(similarities)

# Mostrar la respuesta más similar al usuario
print(f"Mejor respuesta: {answers[index_best_answer]}")
print(f"Similitud: {maximum_similarity}")

Mejor respuesta: For technical support, you can contact us through our web form.
Similitud: 0.695706236471523
